<a href="https://colab.research.google.com/github/hayamaster/Deep-learning/blob/main/Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import models
import torch
import torch.nn as nn
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
import torch.optim as optim

model = models.resnet18(pretrained=True)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1,
1), padding=(1, 1), bias=False)
model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
params_to_update = []
print('Choose learnable parameters')
for name, param in model.named_parameters():
  if name in ['conv1.weight','fc.weight','fc.bias']:
   print(name)
   print(param.requires_grad)
   param.requires_grad = True
   params_to_update.append(param)
else:
   param.requires_grad = False
print('\n')

batch_size = 64
learnning_rate = 0.01
epochs = 10
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

model.to(device)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else()
train_dataset = datasets.MNIST(
   './data', train=True, download=True,
   transform = transforms.Compose([
                              transforms.Resize([112,112]),
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,),
(0.3081,))
   ])
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

test_dataset = datasets.MNIST(
   './data', train=False,
   transform=transforms.Compose([
                            transforms.Resize([112,112]),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,),
(0.3081,))
   ])
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, **kwargs)


print('\n--Result--')
optimizer = optim.Adam(params_to_update, lr = learnning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                 lr_lambda=lambda epoch: 0.8 **
epoch)

loss_function = nn.CrossEntropyLoss()
def train(model, device, train_loader, optimizer, loss_function, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_function(output, target)
    loss.backward()
    optimizer.step()
   
    if batch_idx % 100 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset), 100.
      * batch_idx / len(train_loader), loss.item()
      ))
def test(model, device, test_loader, loss_function):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device) 
      output = model(data)
      test_loss += loss_function(output, target)
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{}({:.0f}%)\n'.format(
test_loss, correct, len(test_loader.dataset), 100. * correct /
len(test_loader.dataset)
  ))

for epoch in range(epochs):
  train(model, device, train_loader, optimizer, loss_function, epoch)
  test(model, device, test_loader, loss_function) 
  scheduler.step()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Choose learnable parameters
conv1.weight
True
fc.weight
True
fc.bias
True




  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



...

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw


--Result--
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.419635
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.288290
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.283337
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.195646
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.095764
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.212494
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.028492
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.118596
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.161399
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.165713

Test set: Average loss: 0.0024, Accuracy: 9546/10000(95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.309338
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.004797
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.175121
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.111711
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.145206
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.093065
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.382034
Train Epoch: 1 